<a href="https://colab.research.google.com/github/edulapalle/CS584-ML/blob/main/ML_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Santosh Reddy Edulapalle**

ML-HW3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading dataset

In [3]:
#required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

path = "/content/drive/MyDrive/A20501739_CollectedData_annotation.csv"
df = pd.read_csv(path)
#head
df.head(10)

,scorer,Unnamed: 1,Unnamed: 2,annotation,annotation.1,annotation.2,annotation.3,annotation.4,annotation.5,annotation.6,...,annotation.26,annotation.27,annotation.28,annotation.29,annotation.30,annotation.31,annotation.32,annotation.33,annotation.34,annotation.35
0,individuals,NaN,NaN,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2
1,bodyparts,NaN,NaN,topleft,topleft,rightdown,rightdown,nose,nose,leftear,...,rightear,rightear,leftHip,leftHip,rightHip,rightHip,tailBase,tailBase,tailEnd,tailEnd
2,coords,NaN,NaN,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.84028879670976,189.79097324447577,338.9385009785614,285.0777848426165,156.75011720291639,220.28275295588077,181.90583546482554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.0779943039246,202.74997962182283,366.38110271882596,348.34822774378193,159.79929517405688,269.0696004941288,174.28289053697426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,232.2172719886438,174.54508338877326,393.06140996630535,230.95487585487263,240.60251140928023,214.18439701359978,242.12710039485046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,131.59439894100723,208.84833556410388,364.0942192404706,335.3892213664348,145.3156998111395,277.45483991476516,160.56158966684202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,labeled-data,23.0,A_male_in_a_new_cage_side_view_4_2022-08-10_15...,460.1433253313964,207.32374657853376,596.5940395399339,319.3810370179473,489.8728105500163,308.7089141189555,540.1842470738346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Handling Null values**

Since our objective is to find the center of the image by taking average of topleft and rightbottom corners, we will simply take all NA values to be 0 so we don't get errors while doing calculations with any null values.

In [23]:
#replacing all NA with 0
df.fillna(0,inplace = True)
#df['annotation.26'].isnull().sum()
df.head(5)

,scorer,Unnamed: 1,Unnamed: 2,annotation,annotation.1,annotation.2,annotation.3,annotation.4,annotation.5,annotation.6,...,annotation.26,annotation.27,annotation.28,annotation.29,annotation.30,annotation.31,annotation.32,annotation.33,annotation.34,annotation.35
0,individuals,0.0,0,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2,mouse2
1,bodyparts,0.0,0,topleft,topleft,rightdown,rightdown,nose,nose,leftear,...,rightear,rightear,leftHip,leftHip,rightHip,rightHip,tailBase,tailBase,tailEnd,tailEnd
2,coords,0.0,0,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0


In [1]:
cols = ["scorer","id","img","m1_topleft_x","m1_topleft_y","m1_rightdown_x","m1_rightdown_y","m1_nose_x","m1_nose_y","m1_leftear_x","m1_leftear_y","m1_rightear_x","m1_rightear_y","m1_leftHip_x","m1_leftHip_y","m1_rightHip_x","m1_rightHip_y","m1_tailBase_x","m1_tailBase_y","m1_tailEnd_x","m1_tailEnd_y","m2_topleft_x","m2_topleft_y","m2_rightdown_x","m2_rightdown_y","m2_nose_x","m2_nose_y","m2_leftear_x","m2_leftear_y","m2_rightear_x","m2_rightear_y","m2_leftHip_x","m2_leftHip_y","m2_rightHip_x","m2_rightHip_y","m2_tailBase_x","m2_tailBase_y","m2_tailEnd_x","m2_tailEnd_y"]

Train-Test-split

Splitting the dataset to 70-30 for training and evaluation.

In [22]:
#top-3 rows are just additional data, we don't require them.
df1 = df[3:]
df1.head()

,scorer,Unnamed: 1,Unnamed: 2,annotation,annotation.1,annotation.2,annotation.3,annotation.4,annotation.5,annotation.6,...,annotation.26,annotation.27,annotation.28,annotation.29,annotation.30,annotation.31,annotation.32,annotation.33,annotation.34,annotation.35
3,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,162.22631573784358,187.5751745878016,319.8845247452017,320.6233217013282,194.86484184217267,196.65162367954176,186.83637763167508,...,0,0,0,0,0,0,0,0,0,0
4,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,268.0451131495447,177.5942613599138,347.3237403991978,240.86470426107923,279.4795305413216,209.61063005688908,287.86476996195796,...,0,0,0,0,0,0,0,0,0,0
5,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.84028879670976,189.79097324447577,338.9385009785614,285.0777848426165,156.75011720291639,220.28275295588077,181.90583546482554,...,0,0,0,0,0,0,0,0,0,0
6,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,146.0779943039246,202.74997962182283,366.38110271882596,348.34822774378193,159.79929517405688,269.0696004941288,174.28289053697426,...,0,0,0,0,0,0,0,0,0,0
7,labeled-data,23.0,A_male_in_a_new_cage_face_view_3_2022-08-10_15...,232.2172719886438,174.54508338877326,393.06140996630535,230.95487585487263,240.60251140928023,214.18439701359978,242.12710039485046,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#splitting dataset
train_df = df.sample(frac=0.7, random_state=0)
test_df = df.drop(train_df.index)

In [17]:
print("training df size: "+str(train_df.shape))
print("testing df size: "+str(test_df.shape))


training df size: (90, 39)
testing df size: (38, 39)
